In [42]:
# 超参数
COLS_GROUPBY = 'holidays'
COLS_TARGET = 'amount'
COLS_ALL = ['date', 'amount', 'refuelLiters', 'order_cnt', 'text_day', 'text_night', 'high', 'low', 'wc_day', 'wd_day', 'wc_night', 'wd_night', 'week', 'holidays', 'encoded_wc_night', 'code_week', 'encoded_holidays', '305_price','encode_label_305price','encode_weight_305price','313_price' ]

# VAULE_Y = 'amount'
import pandas as pd

# 读取excel文件并获取指定列
df = pd.read_csv('Data\结果1_全站按日期分组_2023全年.csv', usecols=COLS_ALL, encoding='GBK')
print(df.tail())


           date  order_cnt       amount  refuelLiters text_day text_night  \
359  2023-12-26    1072045  335929197.6   44738960.24        阴         多云   
360  2023-12-27    1054896  332618128.4   44334481.65       多云          晴   
361  2023-12-28    1032115  323028612.7   42986493.14        晴         多云   
362  2023-12-29    1218840  372592734.2   49432247.01       多云         小雨   
363  2023-12-30    1126207  341635923.3   45312200.19        阴          晴   

     high  low wc_day wd_day  ... encoded_wc_night  wd_night week code_week  \
359     9    0   3~4级     东风  ...                1        东风  星期二         2   
360    12    0    <3级    西北风  ...                0        北风  星期三         3   
361    12    3   3~4级     北风  ...                0        东风  星期四         4   
362    12    5   3~4级    东南风  ...                0       东南风  星期五         5   
363    14    1    <3级    西南风  ...                2       西北风  星期六         6   

     holidays encoded_holidays  305_price  encode_label_305pri

In [43]:
# 构建节假日特征（销量平均值编码）
# 计算每个holidays分类的amount平均值
from sklearn.preprocessing import OrdinalEncoder

# 计算每个holidays分类的amount平均值和计数
holidays_mean = df.groupby(COLS_GROUPBY)[COLS_TARGET].agg(['mean', 'count']).reset_index()
holidays_mean = holidays_mean.rename(columns={'mean': COLS_TARGET+'_mean', 'count': COLS_TARGET+'_count'})

# 对holidays_mean按COLS_GROUPBY排序
holidays_mean = holidays_mean.sort_values(by=COLS_GROUPBY).reset_index(drop=True)

# 将holidays和其平均amount进行合并
df = pd.merge(df, holidays_mean, on=COLS_GROUPBY, how='left')

# 使用OrdinalEncoder对holidays进行编码
encoder = OrdinalEncoder()
df['encoded'] = encoder.fit_transform(df[[COLS_TARGET+'_mean',]])
holidays_mean['encoded'] = encoder.fit_transform(holidays_mean[[COLS_TARGET+'_mean',]])



# 打印前几行结果
print(holidays_mean)


  week   amount_mean  amount_count  encoded
0  星期一  3.719884e+08            52      1.0
1  星期三  3.828688e+08            52      5.0
2  星期二  3.742820e+08            52      2.0
3  星期五  3.923908e+08            52      6.0
4  星期六  3.744019e+08            52      3.0
5  星期四  3.746798e+08            52      4.0
6  星期日  3.502571e+08            52      0.0
